In [61]:
import numpy as np

In [62]:
class Node:
    def __init__(self, frequency, symbol=None):
        self.symbol = symbol
        self.frequency = frequency
        self.left = None
        self.right = None
        self.code = ""

    def __lt__(self, other):
        return self.frequency < other.frequency

    def __eq__(self, other):
        if isinstance(other, Node):
            return self.frequency == other.frequency
        return False

    def is_leaf(self):
        return self.left is None and self.right is None

class CustomList:
    def __init__(self):
        self.data = []

    def pop(self):
        if not self.data:
            raise IndexError("pop from empty list")
        return self.data.pop(0)

    def insert_sorted(self, node):
        for i in range(len(self.data)):
            if self.data[i] > node:
                self.data.insert(i, node)
                return
        self.data.append(node)

    def __len__(self):
        return len(self.data)

class HuffmanTree:
    def __init__(self, symbols, frequencies):
        self.root = self.build_tree(symbols, frequencies)
        self.codebook = {}
        self._generate_codes(self.root, "")
        
    def build_tree(self, symbols, frequencies):
        queue = CustomList()
        for symbol, freq in zip(symbols, frequencies):
            queue.insert_sorted(Node(freq, symbol))

        while len(queue) >= 2:
            left = queue.pop()
            right = queue.pop()
            merged = Node(left.frequency + right.frequency)
            merged.left = left
            merged.right = right
            queue.insert_sorted(merged)

        return queue.pop()

    def _generate_codes(self, node, codeword):
        if node is None:
            return
        if node.is_leaf():
            node.code = codeword
            self.codebook[node.symbol] = node
        else:
            self._generate_codes(node.left, codeword + "1")
            self._generate_codes(node.right, codeword + "0")

    def get_codebook(self):
        return {
            symbol: {"frequency": node.frequency, "code": node.code}
            for symbol, node in self.codebook.items()
        }
    
    def get_inverse_codebook(self):
        return {node["code"]: symbol for symbol, node in self.codebook.items()}
    
    def decode(self, bitstream):
        decoded_symbols = []
        node = self.root

        for bit in bitstream:
            if bit == '1':
                node = node.left
            elif bit == '0':
                node = node.right
            else:
                raise ValueError(f"Bit inválido: {bit}")

            if node.is_leaf():
                decoded_symbols.append(node.symbol)
                node = self.root

        return decoded_symbols

    def decode_with_codebook(self, bitstream):
        inverse_codebook = self.get_inverse_codebook()
        decoded = []
        buffer = ""

        for bit in bitstream:
            buffer += bit
            if buffer in inverse_codebook:
                decoded.append(inverse_codebook[buffer])
                buffer = ""

        if buffer != "":
            raise ValueError("Bitstream inválido: sufixo final não reconhecido")

        return decoded

class HuffmanShaping:
    def __init__(self, symbols, frequencies, suffixes=('00', '01', '11', '10')):
        self.tree = HuffmanTree(symbols, frequencies)
        self.codebook = self.tree.get_codebook()
        self.expanded_codebook = {}
        self._expand_codebook(suffixes)
    
    def get_expanded_codebook(self):
        return dict(sorted(self.expanded_codebook.items()))
    
    def get_inverse_codebook(self):
        return {node["code"]: symbol for symbol, node in self.expanded_codebook.items()}
    
    def _expand_codebook(self, suffixes):
        N = len(self.codebook)
        for symbol, node in self.codebook.items():
            for i, suffix in enumerate(suffixes):
                new_symbol = symbol + i * N
                self.expanded_codebook[new_symbol] = {
                    'code': node["code"] + suffix,
                    'frequency': node["frequency"]
                }
    
    def decode_with_codebook(self, bitstream):
        inverse_codebook = self.get_inverse_codebook()
        decoded = []
        buffer = ""

        for bit in bitstream:
            buffer += bit
            if buffer in inverse_codebook:
                decoded.append(inverse_codebook[buffer])
                buffer = ""

        if buffer != "":
            raise ValueError("Bitstream inválido: sufixo final não reconhecido")

        return decoded
    
def is_prefix_free(codebook):
    codes = [entry['code'] for entry in codebook.values()]
    for i, code1 in enumerate(codes):
        for j, code2 in enumerate(codes):
            if i != j and code2.startswith(code1):
                return False, code1, code2
    return True, None, None
    
def print_huffman_tree(node, level=0):
    if node is not None:
        # Imprime o nó atual com indentação baseada no nível
        print("  " * level + f"{level} Frequência: {node.frequency}")
        
        # Recursivamente imprime os filhos (esquerda e direita)
        print_huffman_tree(node.left, level + 1)
        print_huffman_tree(node.right, level + 1)

#### Exemplo Huffman Shapping - Artigo do Ungerboeck 

##### Dados

In [63]:
symbols = list(range(32))

frequencies = np.array([
    0.03872, 0.02991, 0.02991, 0.02311, 0.01785, 0.01785,
    0.01379, 0.01379, 0.00823, 0.00823, 0.00823, 0.00636, 
    0.00636, 0.00379, 0.00379, 0.00293, 0.00293, 0.00226, 
    0.00226, 0.00175, 0.00135, 0.00135, 0.00081, 0.00081, 
    0.00062, 0.00062, 0.00062, 0.00062, 0.00037, 0.00037, 
    0.00022, 0.00017
])

p_i_n = np.array([
    0.03125, 0.03125, 0.03125, 0.03125,
    0.01563, 0.01563, 0.01563, 0.01563,
    0.00781, 0.00781, 0.00781, 0.00781,
    0.00781, 0.00391, 0.00391, 0.00195, 
    0.00195, 0.00195, 0.00195, 0.00195,
    0.00098, 0.00098, 0.00049, 0.00049, 
    0.00049, 0.00049, 0.00049, 0.00049,
    0.00024, 0.00024, 0.00024, 0.00024
])

##### Huffman Code

In [64]:
tree = HuffmanTree(symbols, frequencies)

codebook = tree.get_codebook()
for s, info in sorted(codebook.items()):
    print(f"{s} | {info['frequency']:.5f} | {info['code']}")

0 | 0.03872 | 000
1 | 0.02991 | 100
2 | 0.02991 | 011
3 | 0.02311 | 111
4 | 0.01785 | 0100
5 | 0.01785 | 0011
6 | 0.01379 | 1100
7 | 0.01379 | 1011
8 | 0.00823 | 10100
9 | 0.00823 | 01011
10 | 0.00823 | 01010
11 | 0.00636 | 11011
12 | 0.00636 | 11010
13 | 0.00379 | 101011
14 | 0.00379 | 101010
15 | 0.00293 | 0010010
16 | 0.00293 | 0010001
17 | 0.00226 | 0010101
18 | 0.00226 | 0010100
19 | 0.00175 | 0010111
20 | 0.00135 | 00100111
21 | 0.00135 | 00100110
22 | 0.00081 | 001000001
23 | 0.00081 | 001000000
24 | 0.00062 | 001011010
25 | 0.00062 | 001011001
26 | 0.00062 | 001011000
27 | 0.00062 | 001000011
28 | 0.00037 | 0010000101
29 | 0.00037 | 0010000100
30 | 0.00022 | 0010110110
31 | 0.00017 | 0010110111


##### Huffman Shaping

In [65]:
tree_shaping = HuffmanShaping(symbols, frequencies)
expand_codebook = tree_shaping.get_expanded_codebook()
for s, info in sorted(expand_codebook.items()):
    print(f"{s} | {info['frequency']:.5f} | {info['code']}")

0 | 0.03872 | 00000
1 | 0.02991 | 10000
2 | 0.02991 | 01100
3 | 0.02311 | 11100
4 | 0.01785 | 010000
5 | 0.01785 | 001100
6 | 0.01379 | 110000
7 | 0.01379 | 101100
8 | 0.00823 | 1010000
9 | 0.00823 | 0101100
10 | 0.00823 | 0101000
11 | 0.00636 | 1101100
12 | 0.00636 | 1101000
13 | 0.00379 | 10101100
14 | 0.00379 | 10101000
15 | 0.00293 | 001001000
16 | 0.00293 | 001000100
17 | 0.00226 | 001010100
18 | 0.00226 | 001010000
19 | 0.00175 | 001011100
20 | 0.00135 | 0010011100
21 | 0.00135 | 0010011000
22 | 0.00081 | 00100000100
23 | 0.00081 | 00100000000
24 | 0.00062 | 00101101000
25 | 0.00062 | 00101100100
26 | 0.00062 | 00101100000
27 | 0.00062 | 00100001100
28 | 0.00037 | 001000010100
29 | 0.00037 | 001000010000
30 | 0.00022 | 001011011000
31 | 0.00017 | 001011011100
32 | 0.03872 | 00001
33 | 0.02991 | 10001
34 | 0.02991 | 01101
35 | 0.02311 | 11101
36 | 0.01785 | 010001
37 | 0.01785 | 001101
38 | 0.01379 | 110001
39 | 0.01379 | 101101
40 | 0.00823 | 1010001
41 | 0.00823 | 0101101
42 | 0

In [66]:
result, prefix, full = is_prefix_free(expand_codebook)

if result:
    print("O código é prefixo-free!")
else:
    print(f"O código '{prefix}' é prefixo de '{full}', então não é prefixo-free.")

O código é prefixo-free!


In [67]:
symbols = np.random.choice(len(frequencies)*4, size=1000, p=np.tile(frequencies, 4)/(frequencies.sum()*4))
print("Símbolos:", list(symbols))

bitstream = ''.join([expand_codebook[i]['code'] for i in symbols])
print("Bits codificados:", bitstream[:100], "...")

decoded2 = tree_shaping.decode_with_codebook(bitstream)
print("Decodificado (codebook):", decoded2)

acuracia = np.mean(symbols == decoded2)
print(f"Acurácia: {acuracia * 100:.2f}%")

Símbolos: [33, 96, 33, 98, 36, 2, 97, 67, 32, 64, 0, 1, 33, 77, 99, 97, 33, 99, 111, 78, 3, 100, 40, 109, 69, 97, 83, 0, 2, 103, 0, 72, 98, 5, 14, 97, 96, 76, 32, 32, 37, 72, 3, 0, 34, 98, 4, 3, 6, 33, 65, 111, 103, 68, 70, 100, 3, 39, 0, 0, 45, 66, 3, 98, 106, 35, 86, 75, 102, 95, 32, 34, 104, 32, 64, 97, 99, 73, 4, 33, 10, 32, 6, 3, 65, 2, 1, 7, 3, 0, 65, 0, 43, 115, 3, 4, 112, 64, 74, 38, 4, 33, 1, 2, 64, 1, 0, 36, 39, 44, 4, 97, 66, 41, 34, 35, 33, 103, 15, 13, 98, 97, 34, 14, 11, 102, 6, 70, 36, 2, 1, 97, 105, 0, 66, 8, 6, 64, 34, 98, 66, 107, 3, 75, 102, 11, 65, 15, 100, 84, 74, 32, 1, 106, 0, 0, 67, 74, 9, 35, 35, 99, 10, 32, 65, 64, 9, 65, 64, 1, 66, 70, 5, 64, 41, 33, 83, 102, 98, 105, 2, 66, 37, 65, 33, 68, 65, 40, 64, 98, 98, 0, 104, 12, 65, 74, 96, 98, 50, 66, 40, 99, 75, 64, 96, 1, 37, 41, 100, 68, 74, 34, 36, 33, 38, 73, 54, 101, 80, 103, 67, 32, 34, 79, 71, 110, 66, 104, 0, 33, 99, 39, 75, 101, 14, 103, 33, 43, 102, 33, 106, 37, 38, 33, 32, 71, 68, 35, 1, 33, 64, 71, 65,

#### Exemplo 3

In [68]:
# exemplo 3
freq = np.array([1/16,1/16,1/16,1/16,1/8,1/8, 1/8,1/8,1/4])
s = range(0,9)
tree = HuffmanTree(s, freq)

codes = tree.get_codebook()
for key in sorted(codes.keys()):
    print(key, codes[key])

print("\n-----------------------\n")
print_huffman_tree(tree.root)

0 {'frequency': 0.0625, 'code': '0011'}
1 {'frequency': 0.0625, 'code': '0010'}
2 {'frequency': 0.0625, 'code': '0001'}
3 {'frequency': 0.0625, 'code': '0000'}
4 {'frequency': 0.125, 'code': '101'}
5 {'frequency': 0.125, 'code': '100'}
6 {'frequency': 0.125, 'code': '011'}
7 {'frequency': 0.125, 'code': '010'}
8 {'frequency': 0.25, 'code': '11'}

-----------------------

0 Frequência: 1.0
  1 Frequência: 0.5
    2 Frequência: 0.25
    2 Frequência: 0.25
      3 Frequência: 0.125
      3 Frequência: 0.125
  1 Frequência: 0.5
    2 Frequência: 0.25
      3 Frequência: 0.125
      3 Frequência: 0.125
    2 Frequência: 0.25
      3 Frequência: 0.125
        4 Frequência: 0.0625
        4 Frequência: 0.0625
      3 Frequência: 0.125
        4 Frequência: 0.0625
        4 Frequência: 0.0625
